In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
from utils.dataset_utils.load_dataset import load_data
from utils.model_utils.load_model import load_model
from utils.model_utils.model_config import ModelConfig
import torch
from tqdm import tqdm
import numpy as np

In [3]:
import copy

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import load_model
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class

In [4]:
from utils.model_utils.evaluate import evaluate_model

In [5]:
model_name = "fabriceyhc/bert-base-uncased-yahoo_answers_topics"
task_type = "classification"
architectures = "bert"
dataset_name = "Yahoo"
num_labels = 10

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import BertModel
from tokenizers import BertWordPieceTokenizer

import torch
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
from tqdm import tqdm
from utils.model_utils.evaluate import get_sparsity

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
checkpoint = None
model_config = ModelConfig(
    model_name=model_name,
    task_type=task_type,
    dataset_name=dataset_name,
    checkpoint=checkpoint,
    device=device,
)

In [9]:
model, tokenizer, checkpoint = load_model(model_config)

Directory /home/Minwoo/LESN/Decompose/DecomposeTransformer/Models/Configs/classification/fabriceyhc/bert-base-uncased-yahoo_answers_topics exists.
Loading the model.
The model fabriceyhc/bert-base-uncased-yahoo_answers_topics is loaded.


In [10]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32
)

Loading the dataset Yahoo
Load cached dataset.
The dataset Yahoo is loaded


In [11]:
def get_sorted_indices(data):

    data_np = np.array(data)
    data_flattened = data_np.flatten()
    sorted_indices = np.argsort(data_flattened)
    row_indices = sorted_indices // 12
    col_indices = sorted_indices % 12


    result = []


    for i in range(len(row_indices)):
        result.append((row_indices[i], col_indices[i]))

    return result
def get_sorted_indices_except_max(data):
    data_np = np.array(data)
    max_indices = np.argmin(data_np, axis=1)
    data_flattened = data_np.flatten()
    sorted_indices = np.argsort(data_flattened)[::-1]
    row_indices = sorted_indices // 12
    col_indices = sorted_indices % 12

    result = []

    for i in range(len(row_indices)):
        # 각 행의 최대값 인덱스를 제외
        if col_indices[i] != max_indices[row_indices[i]]:
            result.append((row_indices[i], col_indices[i]))

    return result


ablating_head_num_in_CI = 100

In [12]:
class_1_data = np.array([
    [-0.07, 0.30, 0.07, -0.10, 0.03, -0.30, 0.13, 0.03, -0.27, -0.53, -0.20, -0.13],
    [-0.20, -0.10, 0.00, 0.00, 0.03, 0.00, -0.20, -0.37, 0.03, 0.27, -0.47, -0.10],
    [0.03, 0.17, 0.07, -0.23, 0.00, 0.10, -0.40, -0.13, -0.17, 0.10, -0.47, -0.03],
    [-1.03, 0.13, 0.13, -0.23, -0.07, 0.07, 0.03, 0.03, -0.20, 0.13, -0.23, -0.27],
    [-1.03, -0.33, -0.20, -0.40, 0.17, -0.10, 0.10, 0.10, -0.13, 0.03, -0.13, -0.13],
    [-0.03, -0.13, 0.07, -0.03, 0.33, 0.00, 0.10, 0.03, -0.27, -0.23, -0.23, -0.17],
    [-0.27, 0.03, -0.10, 0.00, -0.13, -0.33, -0.10, 0.40, 1.03, -0.10, 0.13, 0.23],
    [-0.57, 0.07, 0.10, 0.03, 0.17, 0.70, 0.07, -0.07, -0.57, 0.07, -0.03, 0.10],
    [-0.13, -0.37, -0.03, -0.17, -0.30, 0.03, -0.33, 0.20, 0.13, -0.43, -0.40, -0.50],
    [-0.17, -0.17, 0.13, -0.27, -0.40, 1.77, -0.07, 0.03, 0.40, 0.13, 0.17, -0.30],
    [0.60, -0.30, -0.03, 0.07, -0.03, 0.00, 0.53, -0.03, 0.43, -0.03, 0.13, 0.80],
    [0.50, 0.17, -0.47, -0.10, -0.77, 0.10, 0.60, -0.73, -0.50, -0.03, -0.10, 1.10]
])
class_2_data = np.array([
    [-0.10, -0.07, -0.27, -0.27, 0.27, -0.07, -0.17, -0.33, -0.50, -0.03, -0.17, 0.07],
    [0.07, 0.03, -0.17, 0.10, -0.17, -0.23, -0.17, -0.23, -0.07, -0.33, -0.10, -0.27],
    [-0.17, -0.13, 0.03, 0.10, 0.03, 0.00, -0.13, -0.20, 0.03, -0.07, -0.10, 0.00],
    [0.20, 0.10, 0.07, -0.07, -0.20, -0.43, 0.17, -0.10, -0.37, -0.13, -0.27, -0.07],
    [-0.17, -0.07, 0.10, -0.27, -0.20, 0.00, -0.03, -0.20, 0.00, 0.23, 0.00, 0.10],
    [-0.10, 0.03, 0.07, 0.03, 0.10, -0.10, 0.17, -0.13, -0.23, 0.17, 0.17, -0.03],
    [-0.17, -0.03, 0.13, 0.00, -0.03, -0.13, -0.03, 0.00, -0.47, -0.23, -0.03, 0.30],
    [0.17, 0.03, 0.13, -0.07, -0.23, -0.13, 0.00, 0.00, 0.00, -0.13, 0.03, 0.07],
    [-0.03, -0.13, -0.03, 0.03, -0.03, -0.10, -0.13, -0.27, -0.07, 0.00, 0.07, -0.03],
    [-0.03, -0.70, 0.27, 0.23, 0.10, -1.30, -0.13, -0.10, -0.47, -0.17, 0.03, -0.23],
    [0.00, -0.23, 0.13, 0.17, -0.07, 0.03, -0.57, -0.43, -0.43, 0.00, -0.03, -0.93],
    [-1.53, 0.07, 0.17, 0.03, 0.03, -0.20, -0.60, -0.07, 0.37, 0.37, -0.23, -0.37]
])
class_3_data = np.array([
    [0.23, 0.33, 0.03, -0.10, 0.07, -0.67, 0.17, 0.30, 0.27, -0.43, 0.00, 0.00],
    [0.17, -0.03, 0.20, 0.07, 0.00, 0.00, 0.07, 0.07, 0.20, 0.10, -0.07, 0.07],
    [-0.10, -0.10, 0.20, 0.03, -0.03, -0.07, 0.13, -0.03, -0.03, -0.17, 0.00, 0.10],
    [-0.20, 0.00, -0.13, 0.00, 0.20, 0.10, -0.07, -0.13, -0.03, 0.07, -0.07, -0.20],
    [-0.03, 0.10, -0.20, 0.20, 0.10, -0.27, -0.13, -0.07, -0.07, -0.07, -0.03, 0.03],
    [-0.03, -0.13, -0.07, -0.10, 0.10, 0.03, -0.07, 0.00, 0.13, 0.00, -0.17, 0.00],
    [0.03, -0.17, -0.07, -0.03, -0.03, -0.07, -0.07, -0.10, 0.00, -0.07, 0.03, 0.03],
    [-0.23, -0.13, 0.00, 0.03, 0.07, -0.17, -0.07, 0.00, -0.03, -0.10, -0.20, 0.03],
    [-0.03, -0.10, 0.13, -0.07, 0.03, -0.13, -0.10, 0.07, -0.10, 0.00, -0.33, -0.07],
    [0.30, -0.03, -0.10, 0.13, 0.07, -0.20, -0.07, -0.40, -0.50, -0.03, -0.13, 0.13],
    [0.20, 0.10, -0.20, 0.10, -0.23, 0.07, -1.07, 0.33, 0.27, 0.07, -0.03, -0.20],
    [-0.67, 0.07, 0.00, 0.23, -0.23, 0.23, 0.33, -0.07, -0.17, 0.27, 0.07, -0.83]
])
class_4_data = np.array([
    [0.33, 0.20, -0.07, -0.13, -0.10, 0.10, -0.03, 0.23, -0.17, -0.17, -0.30, 0.00],
    [-0.07, -0.10, 0.33, 0.07, -0.10, -0.13, 0.03, 0.13, 0.03, 0.03, -0.27, 0.23],
    [0.07, -0.43, 0.13, -0.07, 0.10, 0.17, 0.03, -0.13, -0.17, -0.10, 0.07, 0.23],
    [0.50, -0.03, -0.03, 0.20, 0.00, 0.10, -0.17, -0.03, -0.17, -0.03, -0.20, 0.07],
    [-0.17, -0.17, 0.23, -0.07, -0.13, 0.07, -0.23, -0.03, 0.10, 0.13, -0.17, -0.13],
    [-0.07, -0.07, -0.17, 0.00, -0.17, -0.13, -0.07, 0.03, -0.17, 0.10, -0.03, 0.10],
    [0.13, -0.03, -0.17, 0.00, -0.07, -0.07, 0.00, -0.27, 0.30, 0.10, -0.03, 0.07],
    [0.10, -0.10, -0.10, -0.10, 0.03, -0.50, -0.03, 0.00, 0.10, -0.20, -0.23, 0.03],
    [0.07, 0.33, -0.17, 0.10, -0.03, 0.13, 0.03, -0.07, -0.43, -0.13, 0.10, -0.10],
    [0.20, 0.33, -0.17, 0.10, -0.23, -0.87, 0.13, -0.60, -0.07, 0.10, 0.50, -0.10],
    [-0.70, 0.20, -0.03, 0.67, 0.27, 0.03, 1.47, 0.97, -0.07, -0.03, -0.07, 0.27],
    [1.80, 0.30, -0.13, -0.13, 0.57, -0.20, 0.47, 0.03, 0.37, -0.40, 0.03, 0.93]
])
class_5_data = np.array([
    [-0.20, -0.37, -0.10, 0.20, -0.07, 0.33, 0.17, -0.10, 0.03, 0.20, -0.03, 0.03],
    [0.03, 0.03, 0.30, -0.10, -0.23, -0.30, 0.00, -0.27, -0.17, -0.10, 0.17, -0.13],
    [-0.33, 0.03, 0.03, -0.13, -0.10, -0.33, -0.10, -0.17, -0.17, -0.27, 0.07, 0.07],
    [-0.43, -0.10, -0.10, -0.17, -0.13, 0.03, 0.00, -0.10, 0.10, -0.03, 0.13, -0.23],
    [-0.33, 0.13, 0.13, -0.07, 0.10, 0.07, -0.07, -0.07, -0.07, 0.20, -0.03, 0.10],
    [0.10, 0.00, -0.17, -0.13, 0.03, -0.10, -0.03, 0.00, 0.07, 0.03, -0.10, -0.03],
    [0.07, 0.07, -0.17, 0.00, -0.13, 0.17, 0.13, -0.07, 0.20, 0.03, -0.10, 0.00],
    [-0.03, -0.23, 0.13, -0.03, -0.13, -0.37, -0.10, 0.13, -0.07, -0.10, 0.20, -0.20],
    [-0.03, 0.07, 0.00, 0.03, 0.03, -0.03, -0.20, 0.00, 0.10, 0.03, 0.23, 0.27],
    [0.07, 0.57, 0.00, -0.20, -0.13, -1.30, -0.13, 0.20, 0.30, 0.00, -0.53, -0.33],
    [0.63, 0.13, -0.30, -0.63, -0.53, 0.03, -0.67, -0.90, 0.20, 0.17, 0.00, -0.07],
    [-0.57, -0.23, -0.20, 0.30, 0.03, 0.03, -0.73, -0.70, -0.37, 0.60, -0.53, 1.37]
])
class_6_data = np.array([
    [0.10, -0.17, 0.03, 0.00, 0.10, 0.27, -0.10, -0.07, 0.27, 0.23, 0.07, -0.03],
    [0.00, 0.00, 0.03, -0.03, 0.00, -0.03, -0.03, -0.07, 0.13, 0.03, -0.10, -0.23],
    [0.10, 0.07, 0.07, 0.03, 0.07, -0.03, -0.10, 0.07, 0.00, 0.03, 0.00, -0.17],
    [-0.07, 0.00, -0.03, -0.07, 0.03, 0.07, 0.00, -0.10, -0.13, -0.07, -0.07, 0.00],
    [0.03, 0.03, 0.13, 0.03, 0.07, 0.03, -0.07, -0.07, 0.00, 0.00, -0.10, 0.03],
    [-0.03, 0.00, 0.00, 0.00, 0.00, -0.07, -0.03, -0.07, -0.07, -0.10, 0.03, -0.03],
    [0.00, -0.10, 0.00, 0.00, 0.00, -0.03, -0.03, -0.03, -0.17, 0.00, 0.00, 0.03],
    [0.07, -0.07, -0.07, -0.03, -0.10, -0.20, -0.03, 0.03, -0.03, 0.03, 0.03, 0.00],
    [-0.07, 0.03, -0.03, -0.03, -0.07, -0.03, -0.03, -0.03, -0.03, -0.03, -0.03, 0.07],
    [-0.10, -0.27, 0.03, 0.03, 0.00, -0.60, -0.07, 0.13, -0.10, 0.00, -0.03, 0.13],
    [0.00, -0.07, 0.13, 0.03, 0.13, -0.03, -1.00, -0.13, -0.10, 0.00, 0.00, 0.03],
    [-0.10, -0.13, 0.03, -0.07, -0.03, -0.03, 0.13, 0.03, 0.07, 0.03, 0.07, -1.00]
])
class_7_data = np.array([
    [0.20, -0.23, -0.17, -0.27, 0.00, -0.33, 0.07, -0.30, -0.30, -0.17, 0.13, -0.50],
    [-0.13, -0.13, -0.30, 0.17, 0.17, -0.03, -0.07, 0.13, 0.00, -0.10, -0.13, 0.23],
    [-0.30, 0.03, -0.13, 0.13, -0.07, -0.13, -0.10, 0.23, -0.07, -0.20, -0.30, 0.07],
    [-0.13, 0.30, -0.20, 0.03, 0.23, -0.27, -0.17, 0.07, -0.53, -0.07, -0.17, -0.03],
    [-0.43, -0.07, -0.03, -0.13, -0.20, -0.03, -0.03, -0.10, 0.07, 0.27, -0.23, 0.07],
    [0.03, 0.03, -0.10, -0.10, -0.17, -0.10, -0.03, -0.30, 0.10, 0.00, -0.23, -0.20],
    [-0.07, -0.23, -0.10, -0.10, 0.07, -0.27, -0.10, 0.03, -0.30, 0.10, 0.13, 0.07],
    [-0.47, -0.07, -0.07, -0.13, -0.13, -0.13, -0.13, -0.20, 0.13, -0.07, 0.03, -0.13],
    [-0.07, -0.03, -0.03, -0.03, -0.07, 0.00, -0.07, 0.23, -0.17, -0.57, -0.20, -0.67],
    [0.00, 0.07, -0.20, -0.03, 0.03, 0.60, 0.17, -0.17, -0.77, -0.03, -0.07, 0.53],
    [0.03, -0.37, 0.03, -0.30, 0.60, -0.30, 0.53, 0.77, -0.03, -0.20, 0.03, -0.10],
    [0.17, 0.13, 0.13, 0.50, -0.27, -0.27, 0.97, 0.90, 0.03, -0.13, 0.40, -2.33]
])
class_8_data = np.array([
    [0.00, -0.20, -0.23, -0.03, -0.17, 0.27, -0.37, -0.03, -0.17, 0.03, -0.40, -0.03],
    [-0.53, -0.07, 0.03, -0.07, -0.23, -0.23, -0.20, 0.03, -0.10, 0.10, -0.10, 0.33],
    [-0.17, -0.07, -0.10, -0.07, -0.03, -0.07, 0.33, 0.13, 0.20, -0.07, 0.17, -0.13],
    [0.33, -0.10, 0.03, 0.03, -0.17, -0.33, 0.03, 0.00, 0.30, -0.23, -0.17, -0.17],
    [0.27, 0.00, 0.43, -0.30, -0.33, 0.07, -0.13, -0.23, 0.00, -0.07, -0.03, 0.03],
    [-0.07, -0.23, -0.20, 0.13, -0.20, -0.13, 0.03, -0.10, 0.07, 0.00, -0.23, 0.03],
    [-0.20, -0.03, 0.20, 0.00, 0.03, 0.07, -0.13, -0.13, 0.13, -0.17, 0.00, 0.03],
    [-0.10, -0.20, -0.17, 0.13, -0.13, 0.20, -0.07, -0.13, 0.17, 0.03, 0.20, -0.07],
    [-0.03, 0.10, 0.10, 0.00, -0.03, -0.03, 0.13, 0.07, -0.17, 0.10, 0.23, 0.10],
    [0.03, 0.13, 0.00, 0.03, 0.20, 0.60, -0.10, 0.37, 0.57, 0.13, -0.40, 0.03],
    [-0.23, 0.27, -0.30, -0.27, 0.93, 0.10, -0.60, 0.07, 0.00, 0.03, -0.13, 0.00],
    [-0.50, -0.07, -0.13, 0.00, 0.40, 0.20, -0.60, 0.27, -0.13, -0.30, 0.00, 1.37]
])
class_9_data = np.array([
    [-0.40, -0.27, -0.13, -0.07, 0.00, -0.73, -0.37, -0.27, 0.20, -0.53, -0.10, -0.20],
    [0.07, 0.00, 0.07, -0.30, -0.10, -0.13, -0.03, 0.17, -0.30, -0.03, -0.27, -0.07],
    [-0.13, 0.13, -0.10, 0.00, -0.03, -0.23, 0.00, -0.33, -0.10, -0.33, -0.07, -0.07],
    [0.00, -0.17, -0.10, 0.03, -0.03, -0.07, 0.07, -0.20, 0.13, -0.13, -0.20, -0.07],
    [0.37, 0.07, -0.13, 0.10, -0.20, -0.17, 0.07, 0.00, -0.10, -0.20, -0.03, -0.10],
    [-0.13, -0.07, -0.13, -0.03, -0.17, -0.03, -0.17, 0.00, -0.17, -0.07, -0.13, 0.00],
    [0.20, -0.27, -0.27, -0.07, -0.07, -0.17, -0.10, -0.27, -0.43, 0.07, -0.10, -0.07],
    [0.13, -0.07, -0.17, -0.07, -0.20, -0.27, 0.00, 0.03, -0.03, -0.20, -0.13, -0.27],
    [-0.07, 0.07, -0.13, -0.20, 0.03, -0.17, -0.20, -0.27, -0.27, 0.23, 0.37, -0.07],
    [-0.10, -0.03, -0.27, -0.13, -0.07, -0.77, 0.03, 0.03, 0.00, -0.27, -0.13, -0.20],
    [-0.10, -0.30, -0.23, -0.97, -0.93, 0.00, 0.50, -0.33, -0.03, -0.07, -0.10, 0.00],
    [0.40, -0.20, 0.33, -0.43, -0.13, -0.83, -0.93, 0.40, -0.20, 0.07, 0.00, -1.13]
])
class_10_data = np.array([
    [-0.07, 0.07, 0.07, 0.13, 0.00, 0.27, -0.07, 0.10, 0.23, 0.63, -0.07, 0.00],
    [-0.07, -0.07, -0.37, -0.03, -0.13, -0.03, -0.03, -0.13, 0.00, -0.17, -0.07, 0.00],
    [-0.13, 0.20, -0.10, -0.10, 0.10, 0.20, -0.10, 0.07, 0.00, -0.03, 0.20, -0.20],
    [0.03, 0.07, -0.07, 0.10, 0.00, 0.07, -0.10, 0.03, 0.17, 0.00, -0.03, 0.17],
    [-0.10, -0.07, -0.20, 0.13, 0.03, -0.10, 0.03, -0.03, -0.13, -0.13, 0.00, 0.00],
    [0.10, -0.03, -0.07, -0.03, -0.07, 0.00, 0.00, 0.03, -0.03, 0.07, 0.13, -0.13],
    [0.33, -0.10, 0.07, 0.07, -0.10, 0.17, 0.07, -0.03, -0.07, -0.03, 0.03, 0.07],
    [0.63, 0.07, 0.00, -0.07, 0.20, -0.27, -0.07, -0.03, 0.07, 0.17, -0.10, -0.03],
    [-0.07, 0.13, 0.10, 0.00, -0.03, 0.00, 0.03, -0.03, 0.40, 0.33, 0.33, 0.20],
    [-0.20, -0.33, 0.30, 0.07, 0.23, -0.60, 0.00, 0.13, 0.37, -0.07, -0.03, 0.03],
    [-0.07, -0.03, 0.37, 0.20, -0.37, -0.07, 0.90, -1.03, -0.03, 0.00, 0.07, 0.03],
    [-0.43, -0.17, -0.03, -0.07, -0.17, 0.23, -0.13, -0.37, 0.30, -0.07, 0.33, 0.33]
])

class_1_neg_acc = np.array(
[
    [-0.01, -0.09, -0.03, -0.06, -0.04, -0.02, -0.09, -0.06, 0.00, 0.00, -0.03, -0.03],
    [-0.06, -0.05, -0.03, -0.06, -0.05, -0.09, -0.03, -0.04, -0.03, -0.02, -0.01, 0.05],
    [-0.08, -0.06, -0.01, -0.02, 0.00, -0.06, 0.04, -0.04, -0.01, -0.06, 0.03, -0.06],
    [0.13, -0.03, -0.07, -0.01, 0.02, -0.03, 0.01, -0.02, -0.02, -0.02, -0.03, -0.03],
    [0.07, 0.06, 0.07, -0.04, -0.11, -0.03, -0.04, -0.02, -0.02, 0.00, 0.00, 0.00],
    [-0.01, -0.02, -0.05, -0.02, -0.05, -0.08, -0.07, 0.01, 0.00, 0.01, -0.04, 0.01],
    [-0.01, -0.05, -0.06, 0.00, 0.00, 0.01, -0.03, -0.12, -0.15, -0.03, -0.03, 0.04],
    [0.06, -0.04, -0.05, -0.03, -0.07, -0.22, -0.01, -0.01, 0.13, -0.04, 0.00, -0.02],
    [-0.02, 0.09, -0.05, -0.02, 0.03, -0.02, 0.05, -0.05, -0.12, 0.01, 0.10, 0.01],
    [0.04, -0.06, -0.01, 0.05, 0.01, -0.64, -0.05, -0.03, 0.00, -0.05, -0.09, 0.00],
    [-0.09, 0.05, -0.06, -0.06, 0.03, -0.01, -0.22, -0.02, -0.07, -0.03, -0.03, -0.22],
    [-0.20, 0.02, 0.05, 0.02, 0.09, -0.02, -0.25, 0.05, 0.05, -0.03, 0.05, -0.30]
]   
)
class_2_neg_acc = np.array(
[
    [0.05, -0.02, -0.01, 0.05, -0.03, 0.02, -0.01, 0.03, 0.10, -0.04, 0.01, -0.04],
    [-0.01, 0.02, 0.01, -0.02, 0.01, 0.00, 0.00, 0.08, 0.00, 0.04, -0.01, 0.05],
    [0.00, -0.01, 0.01, 0.01, 0.00, -0.02, 0.01, 0.03, 0.00, -0.01, 0.02, 0.02],
    [-0.04, -0.01, -0.02, 0.01, -0.01, 0.03, -0.02, 0.00, 0.04, -0.01, -0.01, 0.00],
    [0.03, 0.00, -0.01, 0.03, 0.00, 0.03, 0.04, 0.02, 0.00, 0.00, -0.02, 0.01],
    [0.00, -0.01, -0.02, -0.02, -0.02, -0.01, 0.00, 0.01, -0.02, -0.02, -0.01, 0.00],
    [0.03, -0.01, 0.00, 0.00, -0.01, 0.01, 0.01, 0.00, 0.05, 0.00, 0.01, -0.01],
    [-0.01, -0.01, -0.02, 0.02, -0.01, 0.00, 0.02, 0.00, -0.01, -0.01, 0.02, -0.01],
    [0.01, 0.01, 0.02, 0.00, 0.02, 0.01, 0.01, 0.02, -0.02, 0.01, -0.01, 0.01],
    [0.00, 0.04, -0.01, 0.01, -0.02, 0.12, 0.03, 0.00, 0.08, 0.03, 0.00, 0.04],
    [0.01, 0.04, -0.02, -0.04, -0.01, 0.02, 0.06, 0.06, 0.04, 0.00, 0.00, 0.13],
    [0.21, 0.03, -0.05, -0.01, 0.02, 0.02, 0.05, 0.02, -0.05, -0.04, 0.02, 0.05]
]
)
class_3_neg_acc = np.array(
[
    [-0.05, -0.04, -0.02, 0.00, 0.04, 0.12, -0.01, -0.04, -0.02, 0.08, -0.02, -0.02],
    [-0.01, 0.00, -0.02, 0.00, -0.01, 0.02, -0.02, 0.02, 0.00, -0.01, -0.01, 0.02],
    [0.02, 0.03, -0.02, 0.02, 0.01, 0.02, -0.01, 0.02, 0.01, 0.02, 0.02, 0.00],
    [0.02, 0.03, 0.05, 0.02, -0.05, -0.04, 0.01, 0.01, 0.01, 0.01, -0.01, 0.02],
    [0.03, -0.02, 0.04, -0.04, -0.01, 0.02, 0.03, 0.03, 0.00, 0.00, 0.02, 0.03],
    [0.00, 0.00, 0.00, 0.04, -0.03, 0.02, 0.02, -0.01, 0.01, 0.06, 0.02, -0.02],
    [0.04, 0.06, 0.04, 0.01, 0.01, 0.07, -0.03, 0.02, 0.02, 0.05, 0.04, 0.05],
    [0.02, 0.02, 0.02, 0.00, -0.01, 0.07, 0.00, -0.01, 0.02, -0.02, 0.06, -0.01],
    [0.00, 0.03, 0.02, 0.02, 0.02, 0.01, -0.02, -0.01, 0.01, 0.06, 0.12, 0.00],
    [-0.05, 0.03, 0.03, -0.05, 0.00, 0.11, 0.03, 0.05, 0.10, 0.04, 0.01, -0.03],
    [-0.04, -0.01, 0.07, -0.03, 0.07, 0.00, 0.28, -0.10, 0.00, 0.00, 0.01, 0.06],
    [0.19, -0.01, -0.01, -0.04, 0.07, -0.07, -0.10, 0.04, 0.05, -0.04, 0.00, 0.30]
]
)
class_4_neg_acc = np.array(
[
    [-0.04, -0.03, 0.01, 0.02, 0.03, -0.09, -0.05, 0.00, 0.02, -0.03, -0.01, 0.02],
    [0.03, 0.02, 0.02, -0.01, -0.01, 0.00, 0.00, -0.04, -0.02, 0.01, -0.01, -0.06],
    [-0.04, 0.00, 0.03, 0.00, 0.00, -0.01, -0.01, 0.01, 0.00, -0.04, -0.04, 0.01],
    [-0.09, 0.02, -0.03, -0.03, -0.02, -0.01, 0.02, 0.00, 0.01, -0.03, -0.04, -0.02],
    [-0.09, -0.01, -0.01, -0.02, -0.01, -0.03, 0.02, -0.05, -0.02, -0.02, -0.02, -0.03],
    [0.01, -0.01, 0.00, -0.01, 0.02, 0.00, 0.02, -0.01, 0.02, 0.01, -0.04, -0.02],
    [-0.03, -0.04, -0.02, -0.01, 0.00, -0.03, 0.00, 0.05, -0.01, -0.04, 0.00, 0.01],
    [0.00, 0.02, 0.02, 0.00, 0.00, 0.05, -0.02, -0.01, -0.03, 0.00, 0.00, -0.03],
    [-0.01, -0.03, 0.03, -0.04, -0.03, -0.01, -0.03, 0.03, 0.08, 0.02, 0.00, 0.03],
    [-0.03, -0.06, 0.03, -0.03, 0.04, 0.14, -0.02, 0.06, -0.05, -0.04, -0.06, 0.05],
    [0.18, -0.08, 0.02, -0.16, -0.06, 0.00, -0.21, -0.20, 0.03, 0.01, 0.01, -0.02],
    [-0.31, -0.06, 0.03, 0.05, -0.09, -0.02, -0.16, -0.01, -0.07, 0.08, 0.02, -0.19]
]  
)
class_5_neg_acc = np.array(
[
    [0.03, 0.04, 0.03, -0.02, -0.01, -0.04, 0.00, 0.00, -0.03, 0.01, -0.01, -0.01],
    [-0.01, 0.00, -0.02, 0.03, 0.00, 0.00, 0.00, 0.02, 0.01, 0.01, -0.02, 0.02],
    [0.02, 0.01, -0.01, -0.01, 0.00, 0.02, -0.01, 0.00, 0.01, 0.02, 0.00, 0.00],
    [0.02, 0.01, 0.01, 0.02, -0.01, 0.00, -0.02, 0.00, -0.01, 0.02, -0.02, 0.02],
    [0.03, -0.01, 0.01, 0.00, 0.02, 0.02, -0.01, 0.01, 0.01, 0.02, 0.02, 0.00],
    [0.00, 0.01, 0.00, 0.01, -0.01, -0.01, 0.02, 0.02, -0.01, 0.00, 0.01, 0.00],
    [0.00, -0.01, 0.01, 0.00, 0.00, -0.02, 0.00, 0.01, 0.01, -0.01, 0.02, 0.01],
    [-0.01, 0.00, 0.00, 0.00, 0.02, 0.03, 0.01, 0.01, -0.03, 0.00, -0.02, 0.02],
    [0.00, 0.00, -0.01, 0.00, 0.00, 0.01, 0.01, 0.00, -0.02, -0.01, 0.00, -0.05],
    [0.03, -0.06, -0.01, -0.01, 0.02, 0.15, 0.00, -0.01, -0.04, 0.00, 0.05, 0.03],
    [-0.07, 0.01, 0.01, 0.05, 0.03, 0.00, 0.13, 0.09, 0.00, 0.00, -0.01, 0.01],
    [0.03, 0.01, 0.01, -0.03, 0.00, 0.00, 0.07, 0.07, 0.02, -0.03, 0.04, -0.13]
] 
)
class_6_neg_acc = np.array(
[
    [-0.01, 0.02, -0.02, 0.00, 0.01, -0.04, 0.02, 0.02, -0.02, -0.02, -0.02, 0.01],
    [0.00, 0.01, 0.00, 0.00, 0.00, 0.01, 0.01, 0.00, -0.02, 0.00, -0.01, 0.00],
    [0.01, 0.01, -0.01, 0.00, 0.00, 0.01, 0.00, -0.02, 0.00, 0.00, 0.01, 0.01],
    [0.00, 0.01, 0.01, 0.00, 0.01, -0.01, 0.01, 0.01, 0.02, -0.01, 0.02, -0.01],
    [-0.03, 0.00, -0.02, 0.03, 0.01, 0.01, 0.01, 0.00, 0.02, 0.01, 0.00, 0.02],
    [0.01, 0.01, 0.00, 0.01, 0.00, 0.01, 0.01, 0.02, -0.02, 0.01, 0.00, 0.01],
    [0.02, 0.01, 0.01, 0.01, 0.01, -0.01, 0.02, 0.00, 0.06, 0.01, 0.01, 0.01],
    [0.01, 0.01, 0.00, 0.02, 0.02, 0.02, 0.01, 0.00, -0.01, 0.01, -0.01, 0.01],
    [0.01, 0.00, 0.01, 0.01, 0.00, 0.00, 0.00, 0.01, 0.00, 0.01, 0.03, -0.01],
    [0.02, 0.03, 0.00, 0.01, -0.01, 0.08, 0.01, 0.00, 0.06, 0.01, 0.01, -0.02],
    [0.02, 0.00, 0.00, 0.01, -0.04, 0.01, 0.16, 0.01, 0.03, 0.01, 0.00, 0.03],
    [0.03, 0.01, 0.03, 0.00, -0.02, -0.01, 0.00, 0.02, 0.00, 0.02, 0.00, 0.14]
]  
)
class_7_neg_acc = np.array(
[
    [-0.02, 0.03, 0.01, 0.01, 0.00, 0.04, 0.00, 0.04, 0.02, 0.06, -0.02, 0.01],
    [-0.01, -0.03, 0.03, 0.01, -0.02, -0.01, -0.01, 0.01, 0.00, 0.00, -0.02, -0.03],
    [0.01, 0.03, 0.02, -0.01, -0.01, -0.02, 0.01, 0.00, 0.00, 0.00, 0.04, 0.00],
    [0.03, -0.02, 0.01, -0.02, -0.01, -0.02, 0.00, 0.00, 0.01, 0.01, 0.02, -0.02],
    [0.03, 0.00, 0.01, 0.00, -0.01, -0.01, -0.01, -0.01, -0.01, 0.00, 0.00, -0.03],
    [-0.04, -0.02, 0.00, 0.00, 0.00, -0.03, 0.00, -0.02, -0.02, -0.02, -0.01, -0.01],
    [0.02, 0.01, 0.00, 0.00, -0.03, 0.04, 0.01, -0.03, 0.00, 0.01, -0.01, 0.00],
    [0.05, -0.02, 0.00, 0.00, 0.03, 0.00, -0.01, 0.01, -0.04, -0.01, -0.01, 0.00],
    [-0.01, -0.02, -0.02, 0.00, -0.01, -0.01, -0.01, -0.07, 0.00, 0.03, 0.02, 0.04],
    [-0.01, 0.00, 0.02, 0.00, -0.01, -0.11, -0.03, 0.07, 0.10, -0.01, -0.02, -0.09],
    [0.00, 0.04, -0.02, -0.03, -0.05, 0.00, -0.03, -0.14, 0.02, 0.03, -0.01, -0.02],
    [-0.08, -0.03, -0.01, -0.06, 0.01, 0.03, -0.15, -0.11, 0.00, 0.00, -0.06, 0.17]
]
)
class_8_neg_acc = np.array(
[
    [0.02, -0.02, -0.02, -0.05, 0.01, -0.10, -0.02, -0.03, -0.01, -0.10, -0.01, 0.00],
    [0.02, 0.00, -0.04, -0.02, 0.00, -0.02, 0.00, -0.08, 0.00, -0.06, -0.01, -0.04],
    [-0.04, 0.00, 0.00, 0.01, 0.00, 0.03, -0.09, -0.06, -0.06, -0.06, -0.13, 0.00],
    [-0.13, -0.03, -0.02, -0.02, 0.02, 0.02, -0.05, -0.05, -0.09, -0.02, -0.09, 0.02],
    [-0.17, 0.00, -0.10, 0.04, 0.04, -0.03, -0.01, -0.03, 0.00, -0.01, -0.01, 0.00],
    [0.01, 0.01, -0.02, 0.00, 0.01, 0.01, 0.00, -0.04, -0.03, -0.03, 0.03, -0.03],
    [-0.02, -0.05, -0.03, -0.02, -0.05, -0.10, -0.01, -0.03, -0.03, -0.01, -0.03, -0.02],
    [-0.01, -0.03, -0.02, -0.03, -0.02, -0.20, -0.04, 0.01, -0.04, 0.01, -0.05, -0.02],
    [-0.01, -0.06, -0.03, 0.00, -0.05, -0.02, -0.07, 0.01, -0.02, -0.06, -0.05, -0.06],
    [-0.02, -0.09, -0.03, -0.03, -0.05, -0.35, 0.03, -0.14, -0.15, -0.08, 0.04, -0.07],
    [0.03, -0.12, 0.00, 0.04, -0.36, -0.02, 0.10, -0.05, -0.03, -0.02, 0.00, 0.00],
    [0.03, -0.02, 0.04, 0.02, -0.15, -0.07, 0.14, -0.08, 0.01, 0.12, 0.01, -0.33]
]
)
class_9_neg_acc = np.array(
[
    [0.05, 0.03, 0.00, -0.04, 0.01, 0.10, 0.10, 0.02, -0.08, 0.06, -0.02, -0.01],
    [-0.03, -0.02, 0.00, 0.04, -0.04, 0.00, -0.02, -0.02, 0.03, 0.01, -0.02, -0.02],
    [0.00, -0.01, -0.02, -0.03, 0.02, 0.01, -0.02, 0.01, 0.00, 0.02, -0.01, -0.02],
    [-0.03, 0.02, 0.00, 0.03, 0.00, 0.01, -0.01, 0.00, -0.03, 0.00, 0.01, -0.04],
    [-0.06, -0.04, 0.01, -0.08, 0.02, -0.03, -0.06, -0.03, -0.01, 0.02, -0.05, -0.01],
    [-0.02, -0.03, 0.01, -0.03, 0.02, -0.02, 0.00, -0.01, 0.02, -0.05, -0.03, 0.00],
    [-0.04, 0.00, 0.00, -0.01, 0.00, 0.01, 0.01, 0.04, 0.05, -0.01, 0.00, 0.01],
    [-0.07, -0.01, 0.01, -0.01, -0.02, 0.03, -0.02, -0.02, -0.01, 0.00, -0.02, 0.01],
    [-0.01, -0.02, 0.00, 0.01, -0.03, 0.00, -0.02, 0.03, 0.06, -0.06, -0.16, 0.00],
    [0.01, 0.00, 0.02, 0.04, -0.02, 0.11, -0.04, -0.01, -0.07, 0.04, 0.00, 0.03],
    [-0.02, 0.02, 0.01, 0.15, 0.17, -0.01, -0.12, 0.06, 0.00, -0.01, 0.04, -0.02],
    [-0.11, 0.01, -0.12, 0.07, 0.00, 0.12, 0.25, -0.10, 0.00, -0.04, -0.05, 0.31]
]
)
class_10_neg_acc = np.array(
[
    [0.01, 0.05, -0.01, 0.03, 0.01, -0.06, 0.01, -0.02, 0.01, -0.09, 0.03, 0.00],
    [0.03, 0.01, 0.08, 0.03, 0.04, -0.02, 0.02, 0.01, 0.03, 0.02, -0.02, 0.03],
    [0.00, 0.00, 0.03, 0.01, 0.02, -0.02, 0.04, 0.01, 0.02, 0.00, 0.02, 0.05],
    [0.02, 0.04, 0.03, 0.00, 0.05, 0.00, 0.05, 0.00, 0.01, 0.01, 0.03, -0.01],
    [-0.02, 0.00, 0.04, 0.00, 0.01, 0.01, 0.00, 0.01, 0.01, 0.05, 0.00, 0.03],
    [0.03, 0.01, 0.02, 0.01, 0.04, 0.04, 0.01, 0.00, 0.01, 0.03, 0.00, 0.02],
    [0.00, 0.01, 0.00, 0.02, 0.04, -0.03, 0.00, 0.04, 0.05, 0.03, 0.01, 0.01],
    [-0.06, 0.00, 0.02, 0.01, 0.00, 0.11, 0.02, 0.01, 0.00, 0.01, 0.02, 0.00],
    [0.01, 0.04, 0.02, 0.01, 0.02, 0.01, 0.00, 0.03, -0.01, -0.06, 0.00, -0.04],
    [0.04, 0.14, -0.03, 0.01, 0.02, 0.10, 0.02, -0.02, -0.02, 0.04, 0.00, 0.05],
    [0.04, -0.02, -0.03, -0.01, 0.21, 0.00, -0.13, 0.23, 0.02, 0.01, 0.00, 0.03],
    [0.12, 0.03, 0.01, 0.02, 0.02, -0.03, 0.10, 0.08, -0.03, 0.03, -0.01, -0.07]
]
)

In [13]:
correct_num = []
correct_num_neg = []
for r in range(20, 20 + 1): # 20개 되살림
    print('recovered head num: ', r)

    # ================================================================================================
    # 1번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_1_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_1_neg_acc) ##############################

    ##########################
    model_1 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_1 = model_1.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_1.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 2번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_2_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_2_neg_acc) ##############################

    ##########################
    model_2 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_2 = model_2.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_2.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 3번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_3_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_3_neg_acc) ##############################

    ##########################
    model_3 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_3 = model_3.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_3.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 4번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_4_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_4_neg_acc) ##############################

    ##########################
    model_4 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_4 = model_4.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_4.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 5번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_5_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_5_neg_acc) ##############################

    ##########################
    model_5 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_5 = model_5.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_5.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #6번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_6_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_6_neg_acc) ##############################

    ##########################
    model_6 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_6 = model_6.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_6.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #7번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_7_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_7_neg_acc) ##############################

    ##########################
    model_7 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_7 = model_7.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_7.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #8번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_8_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_8_neg_acc) ##############################

    ##########################
    model_8 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_8 = model_8.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_8.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #9번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_9_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_9_neg_acc) ##############################

    ##########################
    model_9 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_9 = model_9.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_9.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #10번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_10_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_10_neg_acc) ##############################

    ##########################
    model_10 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_10 = model_10.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_10.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)

recovered head num:  20
actually_recovered_head_num:  20
actually_recovered_head_num:  19
actually_recovered_head_num:  18
actually_recovered_head_num:  19
actually_recovered_head_num:  20
actually_recovered_head_num:  18
actually_recovered_head_num:  20
actually_recovered_head_num:  19
actually_recovered_head_num:  20
actually_recovered_head_num:  19


In [14]:
models = [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9, model_10]

In [15]:
for i, model in enumerate(models):
    print("#Module " + str(i) + " in progress....")
    num_samples = 64
    
    positive_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )
    
    all_samples = sampling_class(
        train_dataloader, 200, 20, num_labels, False, 4, device=device
    )
    
    print("after head prune")
    evaluate_model(model, model_config, test_dataloader)
    
    module = copy.deepcopy(model)
    wr = WeightRemoverBert(model, p=0.9)
    ci = ConcernIdentificationBert(model, p=0.6)
    ti = TanglingIdentification(model, p=0.7)
    
    print("after removing weights")
    
    eval_step = 5
    for idx, batch in enumerate(all_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            wr.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass
    
    print("after CI")
    
    for idx, batch in enumerate(positive_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ci.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass
    
    print("after TI")
    
    for idx, batch in enumerate(negative_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ti.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass
    a, b = get_sparsity(module)
    print(a)
    print(b)
    result = evaluate_model(module, model_config, test_dataloader)

#Module 0 in progress....
after head prune


Evaluating: 100%|██████████| 1875/1875 [05:50<00:00,  5.35it/s]


Loss: 1.0719
Precision: 0.6671, Recall: 0.6588, F1-Score: 0.6594
              precision    recall  f1-score   support

           0       0.53      0.59      0.56      6000
           1       0.74      0.61      0.67      6000
           2       0.70      0.73      0.72      6000
           3       0.52      0.49      0.51      6000
           4       0.79      0.78      0.79      6000
           5       0.88      0.78      0.83      6000
           6       0.53      0.44      0.48      6000
           7       0.52      0.74      0.62      6000
           8       0.69      0.70      0.69      6000
           9       0.75      0.71      0.73      6000

    accuracy                           0.66     60000
   macro avg       0.67      0.66      0.66     60000
weighted avg       0.67      0.66      0.66     60000

after removing weights
after CI
after TI
0.1769633945708363
{'bert.embeddings.word_embeddings.weight': 0.0, 'bert.embeddings.position_embeddings.weight': 0.0, 'bert.embeddings.

Evaluating: 100%|██████████| 1875/1875 [05:54<00:00,  5.29it/s]


Loss: 1.1135
Precision: 0.6606, Recall: 0.6463, F1-Score: 0.6498
              precision    recall  f1-score   support

           0       0.49      0.61      0.54      6000
           1       0.77      0.54      0.63      6000
           2       0.69      0.73      0.71      6000
           3       0.47      0.51      0.49      6000
           4       0.80      0.77      0.78      6000
           5       0.89      0.76      0.82      6000
           6       0.46      0.47      0.47      6000
           7       0.60      0.68      0.64      6000
           8       0.71      0.66      0.69      6000
           9       0.72      0.72      0.72      6000

    accuracy                           0.65     60000
   macro avg       0.66      0.65      0.65     60000
weighted avg       0.66      0.65      0.65     60000

#Module 1 in progress....
after head prune


...

Loss: 1.1423
Precision: 0.6599, Recall: 0.6390, F1-Score: 0.6388
              precision    recall  f1-score   support

           0       0.64      0.46      0.54      6000
           1       0.74      0.60      0.66      6000
           2       0.75      0.65      0.69      6000
           3       0.54      0.45      0.49      6000
           4       0.71      0.83      0.76      6000
           5       0.93      0.73      0.82      6000
           6       0.51      0.45      0.48      6000
           7       0.49      0.74      0.59      6000
           8       0.52      0.81      0.64      6000
           9       0.78      0.67      0.72      6000

    accuracy                           0.64     60000
   macro avg       0.66      0.64      0.64     60000
weighted avg       0.66      0.64      0.64     60000

#Module 9 in progress....
after head prune


Evaluating: 100%|██████████| 1875/1875 [10:51<00:00,  2.88it/s]


Loss: 1.1120
Precision: 0.6573, Recall: 0.6471, F1-Score: 0.6480
              precision    recall  f1-score   support

           0       0.56      0.54      0.55      6000
           1       0.74      0.61      0.67      6000
           2       0.70      0.71      0.71      6000
           3       0.53      0.46      0.49      6000
           4       0.79      0.76      0.78      6000
           5       0.90      0.75      0.82      6000
           6       0.48      0.45      0.46      6000
           7       0.50      0.72      0.59      6000
           8       0.64      0.74      0.69      6000
           9       0.73      0.72      0.72      6000

    accuracy                           0.65     60000
   macro avg       0.66      0.65      0.65     60000
weighted avg       0.66      0.65      0.65     60000

after removing weights
after CI
after TI
0.17758466593970462
{'bert.embeddings.word_embeddings.weight': 0.0, 'bert.embeddings.position_embeddings.weight': 0.0, 'bert.embeddings

Evaluating: 100%|██████████| 1875/1875 [11:27<00:00,  2.73it/s]

Loss: 1.1483
Precision: 0.6476, Recall: 0.6360, F1-Score: 0.6384
              precision    recall  f1-score   support

           0       0.59      0.52      0.55      6000
           1       0.70      0.63      0.66      6000
           2       0.68      0.73      0.70      6000
           3       0.55      0.42      0.48      6000
           4       0.75      0.77      0.76      6000
           5       0.89      0.74      0.81      6000
           6       0.36      0.49      0.41      6000
           7       0.61      0.63      0.62      6000
           8       0.67      0.69      0.68      6000
           9       0.68      0.74      0.71      6000

    accuracy                           0.64     60000
   macro avg       0.65      0.64      0.64     60000
weighted avg       0.65      0.64      0.64     60000



In [16]:
model, tokenizer, checkpoint = load_model(model_config)
evaluate_model(model, model_config, test_dataloader)

Directory /home/Minwoo/LESN/Decompose/DecomposeTransformer/Models/Configs/classification/fabriceyhc/bert-base-uncased-yahoo_answers_topics exists.
Loading the model.
The model fabriceyhc/bert-base-uncased-yahoo_answers_topics is loaded.


Evaluating: 100%|██████████| 1875/1875 [16:54<00:00,  1.85it/s]

Loss: 1.0044
Precision: 0.6874, Recall: 0.6865, F1-Score: 0.6839
              precision    recall  f1-score   support

           0       0.57      0.57      0.57      6000
           1       0.74      0.66      0.69      6000
           2       0.71      0.78      0.74      6000
           3       0.54      0.53      0.53      6000
           4       0.80      0.82      0.81      6000
           5       0.90      0.84      0.87      6000
           6       0.61      0.43      0.50      6000
           7       0.62      0.73      0.67      6000
           8       0.64      0.76      0.70      6000
           9       0.75      0.75      0.75      6000

    accuracy                           0.69     60000
   macro avg       0.69      0.69      0.68     60000
weighted avg       0.69      0.69      0.68     60000



{'loss': 1.0044152938524882,
 'precision': 0.6874371369976651,
 'recall': 0.6865,
 'f1_score': 0.6838781660446563,
 'report': '              precision    recall  f1-score   support\n\n           0       0.57      0.57      0.57      6000\n           1       0.74      0.66      0.69      6000\n           2       0.71      0.78      0.74      6000\n           3       0.54      0.53      0.53      6000\n           4       0.80      0.82      0.81      6000\n           5       0.90      0.84      0.87      6000\n           6       0.61      0.43      0.50      6000\n           7       0.62      0.73      0.67      6000\n           8       0.64      0.76      0.70      6000\n           9       0.75      0.75      0.75      6000\n\n    accuracy                           0.69     60000\n   macro avg       0.69      0.69      0.68     60000\nweighted avg       0.69      0.69      0.68     60000\n'}